In [1]:
#
# SMA Algorithm, Logging, Monitoring
#

import zmq
import numpy as np
import pandas as pd
import datetime as dt
from datetime import date
from tpqoa import tpqoa2
from dotenv import load_dotenv

today = date.today()
log_file = f'SMA_strategy {today}.log'

# sets up the socket communication via ZeroMQ (here: "publisher")
context = zmq.Context()
socket = context.socket(zmq.PUB)

# this binds the socket communication to all IP addresses of the machine
#socket.bind('tcp://0.0.0.0:5555')

# recreating the log file
with open(log_file, 'w') as f:
    f.write('*** NEW LOG FILE ***\n')
    f.write(str(dt.datetime.now()) + '\n\n\n')


def logger_monitor(message, time=True, sep=True):

    '''
    Custom logger and monitor function.
    '''

    with open(log_file, 'a') as f:
        t = str(dt.datetime.now())
        msg = ''
        if time:
            msg += '\n' + t + '\n'
        if sep:
            msg += 80 * '=' + '\n'
        msg += message + '\n\n'
        # sends the message via the socket
        socket.send_string(msg)
        # writes the message to the log file
        f.write(msg)


class SMATrader(tpqoa2.tpqoa2):
    def __init__(self, SMA1=20, SMA2=50, conf_file=None, env_file=None):
        super(SMATrader, self).__init__(conf_file, env_file)
        self.SMA1 = SMA1
        self.SMA2 = SMA2
        self.units = 100000
        self.window = float((SMA1 + SMA2)/2)
        self.position = 0
        self.bar = '5s'
        self.min_length = self.SMA1 + self.SMA2 + 1
        #self.min_length = self.lags + self.window + 1
        self.raw_data = pd.DataFrame()

    def prepare_features(self):
        
        # Other Features
        self.data['return'] = np.log(
            self.data['mid'] / self.data['mid'].shift(1))
        self.data['vol'] = self.data['return'].rolling(self.window).std()
        self.data['mom'] = np.sign(
            self.data['return'].rolling(self.window).mean())
        self.data['Position'] = np.where(self.data['SMA1'] > self.data['SMA2'], 1, -1)
        self.data['Signal'] = self.data['Position'][self.data['Position'].diff() != 0]
        
        # SMA 1 Features
        self.data['SMA1'] = self.data['mid'].rolling(self.SMA1).mean()
        self.data['ema1'] = self.data['mid'].ewm(span=self.SMA1).mean()
        self.data['min1'] = self.data['mid'].rolling(self.SMA1).min()
        self.data['max1'] = self.data['mid'].rolling(self.SMA1).max()
        
        # SMA 2 Features
        self.data['SMA2'] = self.data['mid'].rolling(self.SMA2).mean()
        self.data['ema2'] = self.data['mid'].ewm(span=self.SMA2).mean()
        self.data['min2'] = self.data['mid'].rolling(self.SMA2).min()
        self.data['max2'] = self.data['mid'].rolling(self.SMA2).max()
        
        self.data.dropna(how='any',inplace=True)

    def report_trade(self, pos, order):
        
        '''
        Prints, logs and sends trade data.
        '''

        out = '\n\n' + 80 * '=' + '\n'
        out += '*** GOING {} *** \n'.format(pos) + '\n'
        out += str(order) + '\n'
        out += 80 * '=' + '\n'
        logger_monitor(out)
        print(out)

    def on_success(self, time, bid, ask):
        print(self.ticks, 20 * ' ', end='\r')
        df = pd.DataFrame({'bid': float(bid), 'ask': float(ask)},
                          index=[pd.Timestamp(time).tz_localize(None)])
        self.raw_data = self.raw_data.append(df)
        self.data = self.raw_data.resample(
            self.bar, label='right').last().ffill()
        self.data = self.data.iloc[:-1]


        if len(self.data) > self.min_length:
            logger_monitor('NUMBER OF TICKS: {} | '.format(self.ticks) +
                           'NUMBER OF BARS: {}'.format(self.min_length))
            self.min_length += 1
            self.data['mid'] = (self.data['bid'] + self.data['ask']) / 2
            self.prepare_features()
            features = self.data[self.cols].iloc[-1].values.reshape(1, -1)
            signal = self.data['Signal'][0]
            #print(self.model.predict(features))

            # LOGS AND SENDS MAJOR FINANCIAL INFORMATION
            
            logger_monitor('MOST RECENT DATA\n' +
                           str(self.data[self.cols].tail()),
                           False)

            logger_monitor('features:\n' + str(features) + '\n' +
                           'position: ' + str(self.position) + '\n' +
                           'signal:   ' + str(signal), False)
            
            # LONG

            if self.position in [0, -1] and signal == 1:
                order = self.create_order(self.stream_instrument,
                                          units=(1 - self.position) *
                                          self.units,
                                          suppress=True, ret=True)
                self.report_trade('LONG', order)
                self.position = 1

            # SHORT
            
            elif self.position in [0, 1] and signal == -1:
                order = self.create_order(self.stream_instrument,
                                          units=-(1 + self.position) *
                                          self.units,
                                          suppress=True, ret=True)
                self.report_trade('SHORT', order)
                self.position = -1


            else:  # no trade
                logger_monitor('*** NO TRADE PLACED ***')

            logger_monitor('*** END OF CYCLE ***\n\n', False, False)


#if __name__ == '__main__':
    #smaT = SMATrader(env_file='../.env')
    #smaT.stream_data('SPY')
    #order = smaT.create_order(smaT.stream_instrument,
                             #units=-smaT.p osition * smaT.units,
                             #suppress=True, ret=True)
    #smaT.position = 0
    #smaT.report_trade('NEUTRAL', order)

In [2]:
# Instantiates Trader
smaT = SMATrader(env_file='../.env')

# Instantiates Stream - ticks once when markets are closed..
smaT.stream_data('EUR_USD')